In [1]:
import pandas as pd
from datetime import datetime
from meteostat import Daily, Stations
from geopy.geocoders import Nominatim

def scarica_dati_meteo_citta(nome_citta, start, end, excel = True):
    """
    Scarica i dati meteo giornalieri per una città, trovando automaticamente le coordinate,
    e li salva in un file Excel chiamato 'città.xlsx'.

    Parametri:
    ----------
    nome_citta : str
        Nome della città (es. "Milano", "Roma, Italia")
    start : datetime
        Data di inizio
    end : datetime
        Data di fine

    Ritorna:
    --------
    pd.DataFrame oppure None
    """
    # Geocoding: trova lat/lon della città
    geolocator = Nominatim(user_agent="meteostat_script")
    location = geolocator.geocode(nome_citta)
    if not location:
        print(f"❌ Impossibile trovare la città: {nome_citta}")
        return None

    lat, lon = location.latitude, location.longitude
    print(f"📍 Coordinate trovate per {nome_citta}: {lat:.4f}, {lon:.4f}")

    # Trova stazioni vicine
    stations = Stations().nearby(lat, lon).fetch(5)

    final_df = None
    for i, (station_id, row) in enumerate(stations.iterrows(), start=1):
        print(f"Tentativo {i}: scarico dati da stazione {station_id} ({row['name']})...")
        data = Daily(station_id, start, end).fetch()
        if not data.empty:
            final_df = data
            print(f"✅ Dati trovati per {station_id}")
            break
        else:
            print("⚠️ Nessun dato disponibile per questa stazione.")

    if final_df is not None:
        # Nome file = nome città ripulito (niente spazi o accenti strani)
        file_name = nome_citta.split(",")[0].strip().replace(" ", "_") + ".xlsx"
        sheet_name = nome_citta.split(",")[0].strip()

        if excel:
            final_df.to_excel(file_name, sheet_name=sheet_name)
        print(f"📊 Dati salvati in {file_name} (foglio: {sheet_name})")
        return final_df
    else:
        print("❌ Nessuna stazione vicina ha dati disponibili.")
        return None




In [3]:
cities=["Rome"]

start = datetime(2000, 1, 1)
end = datetime(2024, 12, 31)
for city in cities:
    scarica_dati_meteo_citta(city, start, end, excel = False)


📍 Coordinate trovate per Rome: 41.8933, 12.4829
Tentativo 1: scarico dati da stazione 16235 (Roma / Urbe)...


✅ Dati trovati per 16235
📊 Dati salvati in Rome.xlsx (foglio: Rome)


In [129]:
Dubai= scarica_dati_meteo_citta("Bahrain", start, end, True)

📍 Coordinate trovate per Bahrain: 26.1551, 50.5345
Tentativo 1: scarico dati da stazione 41150 (Bahrain International Airport)...
✅ Dati trovati per 41150
📊 Dati salvati in Bahrain.xlsx (foglio: Bahrain)


In [127]:
print(len(Dubai["tmin"]))
Dubai["tavg"].isna().sum()

Dubai

9114


,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2000-01-01,18.5,16.0,22.1,<NA>,<NA>,<NA>,18.4,<NA>,<NA>,<NA>
2000-01-02,18.9,15.8,22.0,<NA>,<NA>,<NA>,16.1,<NA>,<NA>,<NA>
2000-01-03,19.1,15.5,23.0,<NA>,<NA>,<NA>,11.4,<NA>,<NA>,<NA>
2000-01-04,19.2,13.0,24.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-01-05,20.0,17.4,23.2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...
2024-12-27,19.8,15.0,23.3,0.0,<NA>,<NA>,8.5,<NA>,1021.6,290.0
2024-12-28,20.9,18.0,24.6,0.0,<NA>,<NA>,10.6,<NA>,1019.9,176.0
2024-12-29,21.7,17.2,26.6,0.0,<NA>,<NA>,7.6,<NA>,1019.4,147.0


In [130]:
import pandas as pd

# Assicuriamoci che l'indice sia datetime
Dubai.index = pd.to_datetime(Dubai.index, dayfirst=True)

# Creiamo una serie completa di date dal 01/01/2000 al 31/12/2024
tutte_date = pd.date_range(start='2000-01-01', end='2024-12-31', freq='D')

# Troviamo le date mancanti rispetto all'indice
date_presenti = pd.Series(Dubai.index.unique())
date_mancanti = tutte_date.difference(date_presenti)

# Mostriamo le date mancanti
print("Numero di date mancanti:", len(date_mancanti))
print(date_mancanti)


Numero di date mancanti: 18
DatetimeIndex(['2002-06-18', '2002-06-19', '2003-02-23', '2003-05-28',
               '2003-09-21', '2004-01-22', '2004-09-18', '2005-03-01',
               '2005-07-29', '2005-12-11', '2008-05-21', '2008-05-26',
               '2008-07-20', '2009-03-02', '2012-11-13', '2014-07-12',
               '2017-09-17', '2017-09-18'],
              dtype='datetime64[ns]', freq=None)


In [131]:

# Assicuriamoci che l'indice sia datetime
Dubai.index = pd.to_datetime(Dubai.index, dayfirst=True)

# Creiamo un indice completo di tutte le date
tutte_date = pd.date_range(start='2000-01-01', end='2024-12-31', freq='D')

# Reindicizziamo il DataFrame originale
Buenos_Aires_completo = Dubai.reindex(tutte_date)

# Rinominare l'indice in 'Date' se vuoi
Buenos_Aires_completo.index.name = 'Date'

Buenos_Aires_completo.to_excel('Bahrain.xlsx', index=True)